##  Data Preprocessing

Step 2: 

Perform various transformations on our data as required for efficient and effective processing by Machine Learning algorithms based on the type and needs of individual features, identified through the data explorations performed in Step 1 - Data Exploration - Application Data.

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelBinarizer

# Show all of the data in a dataframe
pd.set_option('display.max_columns', None)

In [2]:
# Training Set
application_train = pd.read_csv('data/application_train.csv')
application_train_SK_ID_CURR = application_train['SK_ID_CURR']
application_train_TARGET = application_train['TARGET']
application_train = application_train.drop('SK_ID_CURR',axis=1)
application_train = application_train.drop('TARGET', axis=1)

print("Loaded Training Set: {0} rows".format(application_train.shape[0]))

application_test = pd.read_csv('data/application_test.csv')
application_test_SK_ID_CURR = application_test['SK_ID_CURR']
application_test = application_test.drop('SK_ID_CURR',axis=1)
print("Loaded Training Set: {0} rows".format(application_test.shape[0]))

Loaded Training Set: 307511 rows
Loaded Training Set: 48744 rows


In [3]:
display(application_train.head())

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [4]:
# Load the list of features indentified in the exploration step which need preprocessing
non_numeric_features = pd.read_csv('data/tmp/non_numeric_features.csv', header=0, index_col=0, names=["feature"])
numeric_features = pd.read_csv('data/tmp/numeric_features.csv', header=0, index_col=0, names=["feature"])
string_to_bool_features = pd.read_csv('data/tmp/string_to_bool.csv', header=0, index_col=0, names=["feature"])
log_transform_features = pd.read_csv('data/tmp/log_transform.csv', header=0, index_col=0, names=["feature"])
#strong_correlations = pd.read_csv('data/tmp/application_strong_correlations.csv', names=["feature", "correlations"])

FileNotFoundError: File b'data/tmp/non_numeric_features.csv' does not exist

In [ ]:
print("Non-Numeric {0}, Numeric {1}, Bool {2}, Log Transform {3}".format(\
        len(non_numeric_features), len(numeric_features), \
        len(string_to_bool_features), len(log_transform_features)))

In [ ]:
'''
Scales a list of numeric features to a range of [0 .. 1] without changing the distribution of the data.  
Accepts a List of column names.  Returns a new dataFrame.
''' 
def scale_features(dataFrame, featureList): 
    dataFrame_transform = pd.DataFrame(data = dataFrame)
    dataFrame_transform[featureList] = scaler.fit_transform(dataFrame_transform[featureList])
    return dataFrame_transform

In [ ]:
''' 
Applies an in-place transformation that converts a Y/N field to binary 1/0
'''
def make_bool(dataFrame, featureName): 
    
    lb = LabelBinarizer()
    bool_name = "BOOL_{0}".format(featureName)    
    dataFrame[bool_name] = lb.fit_transform(dataFrame[featureName])
    
    # This seems to work more consistently than df.drop
    # From: https://stackoverflow.com/questions/43838198/df-drop-is-not-working
    del(dataFrame[featureName])    
    #result = dataFrame.drop(featureName, axis=1) 
    
    return dataFrame

In [ ]:
''' 
Applies an in-place transformation that converts NaN, Inf and -Inf to numeric values
'''
def make_numeric(dataFrame, featureName): 
        
    return np.nan_to_num(dataFrame[featureName])

In [ ]:
'''
Applies an in-place abs() to numeric features
'''
def abs_transform(dataFrame, featureName):     
    
    transformed = dataFrame[featureName].apply(lambda x: abs(x))
        
    dataFrame[featureName] = transformed
    
    return dataFrame

In [ ]:
'''
Applies an in-place log transformation to numeric features
'''
def log_transform(dataFrame, featureName):     
    
    transformed = dataFrame[featureName].apply(lambda x: np.log(x + 1))
    
    transformed_name = "LOG_{0}".format(featureName)

    print("Transformed Name: {0}".format(transformed_name))
    
    dataFrame[transformed_name] = transformed

    # This seems to work more consistently than df.drop
    # From: https://stackoverflow.com/questions/43838198/df-drop-is-not-working
    del(dataFrame[featureName])
    #result = dataFrame.drop(featureName, axis=1)
    
    return dataFrame

In [ ]:
'''
Applies an in-place transformation that offsets all values in a column by the minimum value in the column
'''

def find_offset(dataFrames, featureName): 
    
    offset = 0
    
    for df in dataFrames: 
        if (np.min(df[featureName]) < offset): 
            offset = np.min(df[featureName])

    return abs(offset) + 1
            
            
def offset_negative_values(dataFrame, featureName, offset): 
    print("Offsetting Feature {0} by {1}".format(featureName, offset))
    return dataFrame[featureName].apply(lambda x: (x + offset)), offset

In [ ]:
# Tukey's Method for detecting outliers
# Adapted from the customer segments exercise

def return_outliers(dataFrame, featureList): 
    
    outliers = []
    count = 0
    
    frame_stats = []

    for feature_count, feature in enumerate(featureList): 

        feature_stats = {}

        dataFrame[feature] = np.nan_to_num(dataFrame[feature])

        # Calculate Q1 (25th percentile of the data) for the given feature
        Q1 = np.percentile(dataFrame[feature], 25)

        # Calculate Q3 (75th percentile of the data) for the given feature
        Q3 = np.percentile(dataFrame[feature], 75)

        # TODO: Use the interquartile range to calculate an outlier step
        step = (Q3 - Q1) * 1.5

        feature_stats['Q1_percentile'] = Q1
        feature_stats['Q3_percentile'] = Q3
        feature_stats['step'] = step
        feature_stats['feature'] = feature
        feature_stats['feature_min'] = np.min(dataFrame[feature])
        feature_stats['feature_max'] = np.max(dataFrame[feature])
        feature_stats['feature_median'] = np.median(dataFrame[feature])

        # Calculate the median of the interquartile range
        # TODO: This just sounds good intuitively.  It's probably worth researching the actual best practice.
        feature_stats['IQ_median'] = np.median([Q1, Q3])

        # Save a list of the outliers
        feature_stats['outliers'] = dataFrame.index[~((dataFrame[feature] >= Q1 - step) & (dataFrame[feature] <= Q3 + step))]

        # Just keep a count of the number of fields we identified
        count += len(feature_stats['outliers'])

        frame_stats.append(feature_stats)

    outliers.append(frame_stats)   
        
    print("Total Outliers Identified: {0}".format(count))
        
    return outliers

In [ ]:
def replace_outliers(dataFrame, outliers):
    
    count = 0
    
    for ol in outliers: 

        for feature_count, feature in enumerate(ol): 

            # Some features have infrequent but meaningful values, and averaging to 0 would be a problem.
            # We'll just skip these when they come up.
            # example: "How many credit applications were made in the past hour?" 
            if(feature['IQ_median'] >= 1): 

                featureName = feature['feature']
                print("Processing feature {0}".format(featureName))

                '''
                # Perhaps we can one-shot these?
                median = feature['IQ_median']

                try: 
                    frame.iloc[feature_count, feature['outliers']] = median

                    #df.feature_a.iloc[[1, 3, 15]] = 88

                    print("Replacing {2} values at Feature {0} with {1}".format(featureName, median, len(feature['outliers'])))

                except Exception as e: 
                     print("Skipped Feature {0} because {1}".format(featureName, str(e)))

                '''

                for out_index in feature['outliers']: 

                    try: 
                        median = feature['IQ_median']
                        value = dataFrame.iloc[feature_count, out_index:out_index]

                        # Correct way to set value on the dataframe
                        # Per https://pandas.pydata.org/pandas-docs/version/0.21/indexing.html#indexing-label

                        dataFrame.iloc[feature_count, out_index:out_index] = median

                        #print("Replacing value {2} at index {0} with {1}".format(out_index, median, value))
                    except Exception as e:
                        print("Skipped Record at {1} Index {0} because {2}".format(out_index, featureName, str(e)))

                    # Keep track of the number of values we modified
                    count += 1

    print("Total Outliers Modified: {0}".format(count))
    return dataFrame

## Preprocessing

### Individual Problematic Features

In [ ]:
# There were some instances of NaN in otherwise string-based category fields that were confusing things... 

application_train['FONDKAPREMONT_MODE'] = application_train['FONDKAPREMONT_MODE'].replace({np.nan: "not specified"})
application_test['FONDKAPREMONT_MODE'] = application_test['FONDKAPREMONT_MODE'].replace({np.nan: "not specified"})

application_train['HOUSETYPE_MODE'] = application_train['FONDKAPREMONT_MODE'].replace({np.nan: "not specified"})
application_test['HOUSETYPE_MODE'] = application_test['FONDKAPREMONT_MODE'].replace({np.nan: "not specified"})

application_train['WALLSMATERIAL_MODE'] = application_train['WALLSMATERIAL_MODE'].replace({np.nan: "not specified"})
application_test['WALLSMATERIAL_MODE'] = application_test['WALLSMATERIAL_MODE'].replace({np.nan: "not specified"})

# There are a large number of DATE_EMPLOYED fields with an erroneous value of 365243

# Calculate Q1 (25th percentile of the data) for the given feature
Q1 = np.percentile(application_train['DAYS_EMPLOYED'], 25)

# Calculate Q3 (75th percentile of the data) for the given feature
Q3 = np.percentile(application_train['DAYS_EMPLOYED'], 75)

# We'll just replace this value with the the Inter-Quartile Median
mean_days_employed = np.median([Q1, Q3])

application_train['DAYS_EMPLOYED'] = application_train['DAYS_EMPLOYED'].replace({365243: mean_days_employed})
application_test['DAYS_EMPLOYED'] = application_test['DAYS_EMPLOYED'].replace({365243: mean_days_employed})

# A number of date fields are represented as negative integers, preceding the date of the application.
# This is confusing, and when I look at the correlations, DAYS_BIRTH and DAYS_EMPLOYED end up being indicators 
# of a failure to repay the loan instead of indicators of likelihood to pay the load.  This seems weird, and 
# I'm thinking that using positive values would be a better option.


application_train = abs_transform(application_train, 'DAYS_BIRTH')
application_test = abs_transform(application_test, 'DAYS_BIRTH')

application_train = abs_transform(application_train, 'DAYS_EMPLOYED')
application_test = abs_transform(application_test, 'DAYS_EMPLOYED')

application_train = abs_transform(application_train, 'DAYS_ID_PUBLISH')
application_test = abs_transform(application_test, 'DAYS_ID_PUBLISH')

application_train = abs_transform(application_train, 'DAYS_LAST_PHONE_CHANGE')
application_test = abs_transform(application_test, 'DAYS_LAST_PHONE_CHANGE')

### Convert Y/N String Fields to Boolean

In [ ]:
# Convert any Y/N string fields to boolean
for feature in string_to_bool_features['feature']: 
    print("Making Boolean: {0}".format(feature))
    application_test = make_bool(application_test, feature)
    application_train = make_bool(application_train, feature)

In [ ]:
display(application_test.head())

### One-Hot Encode Non-Numeric Features

In [ ]:
nnf = non_numeric_features['feature']
print(nnf)

# One-Hot Encode all of our non-numeric features
application_test = pd.get_dummies(application_test, columns=nnf)
application_train = pd.get_dummies(application_train, columns=nnf)

In [ ]:
display(application_test.head())
display(application_train.head())

In [ ]:
print("Training Set Columns: {0}".format(application_train.shape[1]))
print("Testing Set Columns: {0}".format(application_test.shape[1]))

# Ensure that train and test sets have the same number of columns
# from https://stackoverflow.com/questions/41335718/keep-same-dummy-variable-in-training-and-testing-data/41339045
application_train,application_test = application_train.align(application_test, join='outer', axis=1, fill_value=0)

print("Aligned Training Set Columns: {0}".format(application_train.shape[1]))
print("Aligned Testing Set Columns: {0}".format(application_test.shape[1]))

### Remove non-numeric values from numeric fields

In [ ]:
# Assemble a list of all numeric fields
numeric = [] 

for feature in log_transform_features['feature']:
    numeric.append("{0}".format(feature))

for feature in numeric_features['feature']: 
    numeric.append("{0}".format(feature))

# Apply the np.nan_to_num transformation
for feature in numeric: 
    application_train[feature] = make_numeric(application_train, feature)
    application_test[feature] = make_numeric(application_test, feature)

### Replace Outliers
We're identifying outliers as 1.5X the Interquartile Range, then replacing them with the value of the Interquartile mean, where the mean is greater than zero.

In [ ]:
outliers_train = return_outliers(application_train, numeric)
outliers_test = return_outliers(application_test, numeric)
application_train = replace_outliers(application_train, outliers_train)
application_test = replace_outliers(application_test, outliers_test)

In [ ]:
display(application_train.head(n=20))
display(application_test.head(n=20))

### Log Transform Skewed Numeric Features

In [ ]:
# These features need log transformations but have negative values, and log() of negative values is undefined.
# We need to offset all the values in the feature such that they're positive. 
# We'll want to find the minimum value across the test and train dataset so that we offset consistently

# NOTE: Rather than do offsets and logs of these, I'm going to try taking the absolute value.

''' 
offset = find_offset([application_train, application_test], 'DAYS_BIRTH')
application_train['DAYS_BIRTH'], offset = offset_negative_values(application_train, "DAYS_BIRTH", offset)
application_test['DAYS_BIRTH'], offset = offset_negative_values(application_test, "DAYS_BIRTH", offset)

offset = find_offset([application_train, application_test], 'DAYS_EMPLOYED')
application_train['DAYS_EMPLOYED'], offset = offset_negative_values(application_train, "DAYS_EMPLOYED", offset)
application_test['DAYS_EMPLOYED'], offset = offset_negative_values(application_test, "DAYS_EMPLOYED", offset)

offset = find_offset([application_train, application_test], 'DAYS_ID_PUBLISH')
application_train['DAYS_ID_PUBLISH'], offset = offset_negative_values(application_train, "DAYS_ID_PUBLISH", offset)
application_test['DAYS_ID_PUBLISH'], offset = offset_negative_values(application_test, "DAYS_ID_PUBLISH", offset)

offset = find_offset([application_train, application_test], 'DAYS_LAST_PHONE_CHANGE')
application_train['DAYS_LAST_PHONE_CHANGE'], offset = offset_negative_values(application_train, "DAYS_LAST_PHONE_CHANGE", offset)
application_test['DAYS_LAST_PHONE_CHANGE'], offset = offset_negative_values(application_test, "DAYS_LAST_PHONE_CHANGE", offset)
'''


In [ ]:
# I'm thinking that log transformations on this data are hurting the scores.  
# It's worth testing out.... 

'''
for feature in log_transform_features['feature']: 
    application_train = log_transform(application_train, feature)
    application_test = log_transform(application_test, feature)   
'''

### Scale Numeric Fields

In [ ]:
# Initialize a scaler, then apply it to the features
scaler = StandardScaler()

# Technique for selecting numeric fields from: 
# https://stackoverflow.com/questions/25039626/how-do-i-find-numeric-columns-in-pandas
numeric = list(application_test.select_dtypes(include=[np.number]).columns.values)

scaler = scaler.fit(application_train)

display(numeric)

In [ ]:
application_train[numeric] = scaler.transform(application_train[numeric])
application_test[numeric] = scaler.transform(application_test[numeric])

### Preview
Show our transformed dataset

In [ ]:
# Show an example of a record with scaling applied
display(application_train.head())
display(application_test.head())

### Output
Save out our preprocessed data to temporary intermediate files

In [ ]:
application_train['SK_ID_CURR'] = application_train_SK_ID_CURR
application_train['TARGET'] = application_train_TARGET
application_test['SK_ID_CURR'] = application_test_SK_ID_CURR

application_train.to_csv('data/tmp/application_train_preprocessed_outliers_removed_abs_no_log.csv')
application_test.to_csv('data/tmp/application_test_preprocessed_outliers_removed_abs_no_log.csv')